In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

In [2]:
class WeatherClassifier(nn.Module):
    def __init__(self, num_classes):
        super(WeatherClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 56 * 56, 256)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x



In [5]:
class WeatherImageClassifier:
    def __init__(self, data_path, num_classes, test_size=0.2, batch_size=32, num_epochs=10, learning_rate=0.001):
        self.data_path = data_path
        self.num_classes = num_classes
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        full_dataset = ImageFolder(self.data_path, transform=self.transform)
        train_size = int((1.0 - self.test_size) * len(full_dataset))
        test_size = len(full_dataset) - train_size
        self.train_dataset, self.test_dataset = random_split(full_dataset, [train_size, test_size])

        self.train_loader = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)
        self.test_loader = DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

        self.model = WeatherClassifier(self.num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def train(self):
        for epoch in range(self.num_epochs):
            train_loss = test_loss = 0.0
            correct = total = 0
            self.model.train()
            for images, labels in self.train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                train_loss += loss.item()
            train_loss /= len(self.train_loader)
            self.model.eval()
            with torch.no_grad():
                for images, labels in self.test_loader:
                    outputs = self.model(images)
                    loss = self.criterion(outputs, labels)
                    test_loss += loss.item()
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            test_loss /= len(self.test_loader)
            accuracy = 100 * correct / total

            print(f'Epoch [{epoch+1}/{self.num_epochs}], '
                  f'Train Loss: {train_loss:.4f}, '
                  f'Test Loss: {test_loss:.4f}, '
                  f'Test Accuracy: {accuracy:.2f}%')

classifier = WeatherImageClassifier(data_path='dataset', num_classes=11)
classifier.train()

Epoch [5/10], Train Loss: 0.0901, Test Loss: 1.0871, Test Accuracy: 74.00%
Epoch [6/10], Train Loss: 0.0445, Test Loss: 1.2992, Test Accuracy: 70.87%
Epoch [7/10], Train Loss: 0.0432, Test Loss: 1.3367, Test Accuracy: 72.47%
Epoch [8/10], Train Loss: 0.0185, Test Loss: 1.3783, Test Accuracy: 72.91%
Epoch [9/10], Train Loss: 0.0188, Test Loss: 1.4384, Test Accuracy: 72.10%
Epoch [10/10], Train Loss: 0.0093, Test Loss: 1.4953, Test Accuracy: 73.78%


In [9]:
classifier.model.eval()

WeatherClassifier(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=100352, out_features=256, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=256, out_features=11, bias=True)
)

In [10]:
model_path = 'model.pth'
torch.save(classifier.model.state_dict(), model_path)

In [11]:
from PIL import Image

In [14]:
classes_map = {0:'dew', 1: 'fogsmog', 2: 'frost', 3: 'glaze', 4:'hail', 5:'lightning', 6: 'rain', 7:'rainbow', 8:'rime', 9: 'sandstorm', 10:'snow'}

In [15]:
model_path = 'model.pth'

# Load the model architecture
model = WeatherClassifier(num_classes=11)

# Load the trained model state dict
model.load_state_dict(torch.load(model_path))

# Ensure the model is in evaluation mode
model.eval()

# Define the transformations for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess the sample image
image_path = 'dataset/snow/0830.jpg'
image = Image.open(image_path)
input_tensor = transform(image).unsqueeze(0)

# Perform prediction
with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)
    print("Predicted class:", classes_map[predicted.item()])

Predicted class: snow
